# Projet Final Apache Spark

**Nom Etudiant :**  Gueye

**Prenom Etudiant:**  Serigne Mor Talla

**Classe :**  Master I BDA


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public ou privé) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Jeudi 10 janvier 2021** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [3]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [4]:
!head -1 "sf-fire-calls.csv"

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [5]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...


In [6]:
val sfFireFile = "sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [7]:
fireDF.cache()

res0: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [8]:
fireDF.count()

res1: Long = 175296


In [9]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [10]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [11]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

In [12]:
// Reponse 1: 29
val distinctTypesOfCallsDF = fireDF.select("CallType").distinct()
    distinctTypesOfCallsDF.count()

distinctTypesOfCallsDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string]
res5: Long = 30


### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [13]:
// Reponse 2
distinctTypesOfCallsDF.collect().foreach(println)

[Elevator / Escalator Rescue]
[Marine Fire]
[Aircraft Emergency]
[Confined Space / Structure Collapse]
[Administrative]
[Alarms]
[Odor (Strange / Unknown)]
[Citizen Assist / Service Call]
[HazMat]
[Watercraft in Distress]
[Explosion]
[Oil Spill]
[Vehicle Fire]
[Suspicious Package]
[Extrication / Entrapped (Machinery, Vehicle)]
[Other]
[Outside Fire]
[Traffic Collision]
[Assist Police]
[Gas Leak (Natural and LP Gases)]
[Water Rescue]
[Electrical Hazard]
[High Angle Rescue]
[Structure Fire]
[Industrial Accidents]
[Medical Incident]
[Mutual Aid / Assist Outside Agency]
[Fuel Spill]
[Smoke Investigation (Outside)]
[Train / Rail Incident]


### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

In [14]:
// Reponse 3   >   <: select("CallType","ResponseDelayedinMins").where($"ResponseDelayedinMins" > 5).show(10,false)
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
//bon code
/* newFireDF.
select("CallType","ResponseDelayedinMins").where($"ResponseDelayedinMins" > 5).
collect().
foreach(println) */
// pour voir le resultat
newFireDF.select("CallType","ResponseDelayedinMins").where($"ResponseDelayedinMins" > 5).show(10,false)

+-----------------------------+---------------------+
|CallType                     |ResponseDelayedinMins|
+-----------------------------+---------------------+
|Medical Incident             |5.35                 |
|Medical Incident             |6.25                 |
|Medical Incident             |5.2                  |
|Citizen Assist / Service Call|5.6                  |
|Medical Incident             |7.25                 |
|Medical Incident             |11.916667            |
|Medical Incident             |5.116667             |
|Medical Incident             |8.633333             |
|Medical Incident             |95.28333             |
|Medical Incident             |5.45                 |
+-----------------------------+---------------------+
only showing top 10 rows



newFireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


### Transformations des dates
Maintenant nous allons d'abord:
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard
2. Retourner le Dataframe transformée
3. Mettre en cache le nouveau DataFrame

In [15]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res8: fireTSDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 4
**Quels sont les types d'appels les plus courants?**

In [16]:
//Reponse 4: les appels les plus courants sont ceu de "Medical Incident"

val lesDifferentAppelNbreDecrois=fireTSDF.
groupBy("CallType").
count().
orderBy(desc("count"))

lesDifferentAppelNbreDecrois.
select("CallType").
show(1,false)




+----------------+
|CallType        |
+----------------+
|Medical Incident|
+----------------+
only showing top 1 row



lesDifferentAppelNbreDecrois: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string, count: bigint]


### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [17]:
//Reponse 5-a
val lesBoxAppelPlusCourants=fireTSDF
lesBoxAppelPlusCourants.
select("Box").
where($"CallType" === "Medical Incident").distinct().
collect().
foreach(println)

[2136]
[5645]
[0543]
[7711]
[7252]
[6613]
[1512]
[3210]
[1436]
[2162]
[7273]
[3414]
[8433]
[5613]
[3441]
[7655]
[7362]
[0833]
[6248]
[5657]
[5523]
[2275]
[7328]
[7743]
[0905]
[6366]
[1241]
[3526]
[7263]
[1361]
[1265]
[0558]
[8832]
[0861]
[8621]
[7616]
[6143]
[3121]
[3266]
[5234]
[8153]
[6252]
[0724]
[4236]
[6227]
[8145]
[3652]
[8583]
[6464]
[7316]
[8179]
[4464]
[7544]
[5134]
[7541]
[0355]
[1445]
[9911]
[4262]
[0932]
[5434]
[1425]
[6264]
[6217]
[4465]
[3553]
[5543]
[7364]
[0312]
[0545]
[6212]
[4315]
[3656]
[1418]
[3365]
[3641]
[0314]
[8824]
[7353]
[0970]
[7121]
[4333]
[6612]
[2144]
[7434]
[1435]
[6333]
[5412]
[7256]
[3661]
[8483]
[2341]
[4415]
[8682]
[3454]
[8432]
[3145]
[5352]
[5377]
[8547]
[2436]
[1412]
[2545]
[8424]
[8173]
[8662]
[6423]
[7625]
[5732]
[6121]
[7521]
[7621]
[2931]
[8714]
[5426]
[0223]
[4165]
[4564]
[8421]
[2951]
[6632]
[6147]
[6721]
[1236]
[2431]
[2115]
[4546]
[6245]
[5546]
[7335]
[7652]
[6621]
[8257]
[3521]
[1235]
[7337]
[1624]
[0742]
[1112]
[7134]
[8511]
[8743]
[7334]

[1244]
[2175]
[8147]
[6335]
[6457]
[2413]
[6542]
[2574]
[8565]
[7651]
[5423]
[1321]
[2641]
[5476]
[3217]
[5278]
[1264]
[8484]
[8752]
[7173]
[1363]
[7632]
[7713]
[3232]
[6233]
[2352]
[1561]
[0552]
[3253]
[4434]
[0931]
[6496]
[2212]
[5222]
[3542]
[0902]
[7411]
[0413]
[3616]
[0613]
[6272]
[6157]
[3349]
[2425]
[6341]
[1163]
[2614]
[5524]
[7225]
[4214]
[8146]
[0668]
[8774]
[0448]
[4612]
[3612]
[0937]
[8277]
[7613]
[5631]
[5646]
[6263]
[4156]
[3564]
[1646]
[0922]
[9116]
[6495]
[5237]
[5451]
[8855]
[8853]
[7554]
[8664]
[2553]
[1162]
[8561]
[4454]
[3533]
[5357]
[6516]
[6146]
[6551]
[0556]
[6412]
[4424]
[8434]
[8655]
[8252]
[6646]
[5465]
[7617]
[5674]
[2225]
[5175]
[5375]
[4143]
[9955]
[6462]
[5227]
[5455]
[3245]
[8134]
[8163]
[6633]
[8254]
[5341]
[7535]
[4625]
[3455]
[4252]
[7635]
[8375]
[7274]
[6377]
[3367]
[1612]
[5125]
[6165]
[8646]
[8515]
[8315]
[6215]
[8452]
[0994]
[8171]
[4367]
[7226]
[5225]
[2624]
[7571]
[3362]
[1225]
[8117]
[5736]
[7454]
[6274]
[5445]
[6547]
[1413]
[7511]
[2141]
[7144]

lesBoxAppelPlusCourants: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 5-a
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [18]:
//Reponse 5-b
//box=94102 et 94103 quartier san Francisco  San Francisco
val districtSanFranciscoZipcode=fireTSDF
districtSanFranciscoZipcode.
select("City","Zipcode","Address").
where($"Zipcode" === 94102 or $"Zipcode" === 94103 and $"City" === "San Francisco").
collect().
foreach(println)

[San Francisco,94103,6TH ST/MISSION ST]
[San Francisco,94103,15TH ST/MISSION ST]
[San Francisco,94103,900 Block of MARKET ST]
[San Francisco,94103,1300 Block of HOWARD ST]
[San Francisco,94103,300 Block of CLEMENTINA ST]
[San Francisco,94102,100 Block of TURK ST]
[San Francisco,94103,0 Block of 8TH ST]
[San Francisco,94103,JESSIE ST/6TH ST]
[San Francisco,94103,0 Block of 6TH ST]
[San Francisco,94103,100 Block of 6TH ST]
[San Francisco,94103,0 Block of SPENCER ST]
[San Francisco,94103,0 Block of SPENCER ST]
[San Francisco,94102,TURK ST/JONES ST]
[San Francisco,94102,0 Block of MCALLISTER ST]
[San Francisco,94103,7TH ST/HOWARD ST]
[San Francisco,94102,400 Block of EDDY ST]
[San Francisco,94102,500 Block of GOLDEN GATE AVE]
[San Francisco,94103,1100 Block of HOWARD ST]
[San Francisco,94102,LAGUNA ST/GOLDEN GATE AV]
[San Francisco,94103,800 Block of MARKET ST]
[San Francisco,94102,900 Block of MARKET ST]
[San Francisco,94103,0 Block of 6TH ST]
[San Francisco,94103,0 Block of 6TH ST]
[San 

[San Francisco,94102,100 Block of EDDY ST]
[San Francisco,94102,400 Block of GROVE ST]
[San Francisco,94102,LEAVENWORTH ST/GOLDEN GATE AV]
[San Francisco,94102,200 Block of EDDY ST]
[San Francisco,94103,100 Block of SOUTH VAN NESS AVE]
[San Francisco,94102,900 Block of MARKET ST]
[San Francisco,94102,300 Block of EDDY ST]
[San Francisco,94102,200 Block of EDDY ST]
[San Francisco,94103,1200 Block of MARKET ST]
[San Francisco,94102,800 Block of MARKET ST]
[San Francisco,94102,200 Block of LEAVENWORTH ST]
[San Francisco,94103,1000 Block of BRYANT ST]
[San Francisco,94102,200 Block of GOLDEN GATE AVE]
[San Francisco,94103,0 Block of 6TH ST]
[San Francisco,94103,13TH ST/SOUTH VAN NESS AV]
[San Francisco,94103,100 Block of ERIE ST]
[San Francisco,94103,200 Block of 9TH ST]
[San Francisco,94103,200 Block of 9TH ST]
[San Francisco,94103,16TH ST/MISSION ST]
[San Francisco,94103,4TH ST/MISSION ST]
[San Francisco,94103,800 Block of BRANNAN ST]
[San Francisco,94103,100 Block of 7TH ST]
[San Franci

[San Francisco,94103,400 Block of 6TH ST]
[San Francisco,94103,16TH ST/MISSION ST]
[San Francisco,94102,300 Block of JONES ST]
[San Francisco,94102,100 Block of EDDY ST]
[San Francisco,94103,POWELL ST/MARKET ST]
[San Francisco,94103,1300 Block of HOWARD ST]
[San Francisco,94102,200 Block of LEAVENWORTH ST]
[San Francisco,94103,14TH ST/MISSION ST]
[San Francisco,94102,0 Block of MCALLISTER ST]
[San Francisco,94103,MISSION ST/15TH ST]
[San Francisco,94103,800 Block of BRYANT ST]
[San Francisco,94102,1200 Block of MARKET ST]
[San Francisco,94102,GOLDEN GATE AV/MARKET ST]
[San Francisco,94103,8TH ST/MARKET ST]
[San Francisco,94102,0 Block of STOCKTON ST]
[San Francisco,94103,0 Block of 6TH ST]
[San Francisco,94102,JONES ST/EDDY ST]
[San Francisco,94102,0 Block of UNITED NATIONS PLZ]
[San Francisco,94102,200 Block of JONES ST]
[San Francisco,94103,16TH ST/MISSION ST]
[San Francisco,94103,FOLSOM ST/16TH ST]
[San Francisco,94102,300 Block of GEARY ST]
[San Francisco,94103,400 Block of 9TH ST]

[San Francisco,94103,3RD ST/MARKET ST]
[San Francisco,94102,700 Block of VAN NESS AVE]
[San Francisco,94103,400 Block of 6TH ST]
[San Francisco,94103,STOCKTON ST/MARKET ST]
[San Francisco,94103,100 Block of OTIS ST]
[San Francisco,94103,10TH ST/MISSION ST]
[San Francisco,94103,2800 Block of 16TH ST]
[San Francisco,94102,300 Block of JONES ST]
[San Francisco,94102,800 Block of MARKET ST]
[San Francisco,94102,600 Block of LAGUNA ST]
[San Francisco,94102,100 Block of EDDY ST]
[San Francisco,94103,1000 Block of MISSION ST]
[San Francisco,94103,1100 Block of MISSION ST]
[San Francisco,94103,500 Block of 9TH ST]
[San Francisco,94102,600 Block of LAGUNA ST]
[San Francisco,94103,2300 Block of 16TH ST]
[San Francisco,94102,0 Block of LARKIN ST]
[San Francisco,94102,100 Block of FRANKLIN ST]
[San Francisco,94103,9TH ST/BRYANT ST]
[San Francisco,94103,800 Block of MARKET ST]
[San Francisco,94103,0 Block of 6TH ST]
[San Francisco,94103,5TH ST/HOWARD ST]
[San Francisco,94102,CYRIL MAGNIN ST/EDDY ST

[San Francisco,94102,500 Block of POST ST]
[San Francisco,94102,200 Block of OFARRELL ST]
[San Francisco,94102,500 Block of OFARRELL ST]
[San Francisco,94102,300 Block of GEARY ST]
[San Francisco,94102,0 Block of TURK ST]
[San Francisco,94102,0 Block of MASON ST]
[San Francisco,94103,1500 Block of MISSION ST]
[San Francisco,94103,MARKET ST/DOLORES ST]
[San Francisco,94103,MARKET ST/7TH ST]
[San Francisco,94102,0 Block of TURK ST]
[San Francisco,94103,1000 Block of MARKET ST]
[San Francisco,94103,0 Block of 4TH ST]
[San Francisco,94103,15TH ST/SOUTH VAN NESS AV]
[San Francisco,94103,300 Block of CLEMENTINA ST]
[San Francisco,94102,OFARRELL ST/TAYLOR ST]
[San Francisco,94103,MASON ST/MARKET ST]
[San Francisco,94103,MASON ST/MARKET ST]
[San Francisco,94103,1500 Block of 15TH ST]
[San Francisco,94102,300 Block of ELLIS ST]
[San Francisco,94102,ELLIS ST/MASON ST]
[San Francisco,94102,100 Block of EDDY ST]
[San Francisco,94103,POWELL ST/MARKET ST]
[San Francisco,94102,0 Block of TURK ST]
[Sa

...

[San Francisco,94103,100 Block of DUBOCE AVE]
[San Francisco,94103,8TH ST/FOLSOM ST]
[San Francisco,94103,LAFAYETTE ST/HOWARD ST]
[San Francisco,94103,1000 Block of FOLSOM ST]
[San Francisco,94102,900 Block of MARKET ST]
[San Francisco,94102,TURK ST/TAYLOR ST]
[San Francisco,94103,1500 Block of MISSION ST]
[San Francisco,94103,1100 Block of MISSION ST]
[San Francisco,94102,400 Block of TURK ST]
[San Francisco,94103,800 Block of MISSION ST]
[San Francisco,94103,100 Block of OTIS ST]
[San Francisco,94102,300 Block of ELLIS ST]
[San Francisco,94102,600 Block of LAGUNA ST]
[San Francisco,94102,300 Block of GOLDEN GATE AVE]
[San Francisco,94103,800 Block of MARKET ST]
[San Francisco,94103,16TH ST/VERMONT ST]
[San Francisco,94103,1700 Block of MARKET ST]
[San Francisco,94103,4TH ST/MARKET ST]
[San Francisco,94102,1100 Block of MARKET ST]
[San Francisco,94103,6TH ST/MISSION ST]
[San Francisco,94103,0 Block of BLK LEAVENWORTH ST]
[San Francisco,94102,TURK ST/TAYLOR ST]
[San Francisco,94103,800

[San Francisco,94103,1200 Block of MARKET ST]
[San Francisco,94102,600 Block of POLK ST]
[San Francisco,94102,600 Block of POLK ST]
[San Francisco,94102,600 Block of JONES ST]
[San Francisco,94103,100 Block of OTIS ST]
[San Francisco,94102,300 Block of EDDY ST]
[San Francisco,94103,1100 Block of MISSION ST]
[San Francisco,94103,700 Block of MARKET ST]
[San Francisco,94102,300 Block of ELLIS ST]
[San Francisco,94103,STEVENSON ST/6TH ST]
[San Francisco,94103,0 Block of 6TH ST]
[San Francisco,94103,1000 Block of HOWARD ST]
[San Francisco,94102,0 Block of GROVE ST]
[San Francisco,94102,600 Block of SUTTER ST]
[San Francisco,94102,600 Block of SUTTER ST]
[San Francisco,94103,STEVENSON ST/6TH ST]
[San Francisco,94102,400 Block of ELLIS ST]
[San Francisco,94102,100 Block of EDDY ST]
[San Francisco,94102,100 Block of MASON ST]
[San Francisco,94103,16TH ST/VALENCIA ST]
[San Francisco,94102,200 Block of BUCHANAN ST]
[San Francisco,94102,0 Block of JONES ST]
[San Francisco,94103,900 Block of HOWA

[San Francisco,94103,1500 Block of MISSION ST]
[San Francisco,94102,100 Block of EDDY ST]
[San Francisco,94103,200 Block of 8TH ST]
[San Francisco,94102,100 Block of JONES ST]
[San Francisco,94102,0 Block of MCALLISTER ST]
[San Francisco,94103,0 Block of 5TH ST]
[San Francisco,94102,0 Block of BLK SOUTH VAN NESS AV]
[San Francisco,94103,1100 Block of MARKET ST]
[San Francisco,94102,800 Block of MARKET ST]
[San Francisco,94103,MARKET ST/5TH ST]
[San Francisco,94102,400 Block of GOLDEN GATE AVE]
[San Francisco,94102,200 Block of GOLDEN GATE AVE]
[San Francisco,94103,1100 Block of MISSION ST]
[San Francisco,94102,800 Block of MARKET ST]
[San Francisco,94102,6TH ST/MARKET ST]
[San Francisco,94102,200 Block of TURK ST]
[San Francisco,94103,500 Block of 6TH ST]
[San Francisco,94102,100 Block of LARKIN ST]
[San Francisco,94102,200 Block of LEAVENWORTH ST]
[San Francisco,94103,0 Block of 6TH ST]
[San Francisco,94103,400 Block of VALENCIA ST]
[San Francisco,94102,0 Block of CYRIL MAGNIN ST]
[Sa

[San Francisco,94103,MISSION ST/16TH ST]
[San Francisco,94102,100 Block of OFARRELL ST]
[San Francisco,94102,0 Block of MASON ST]
[San Francisco,94102,0 Block of MCALLISTER ST]
[San Francisco,94102,200 Block of MCALLISTER ST]
[San Francisco,94103,800 Block of HOWARD ST]
[San Francisco,94103,VALENCIA ST/CLINTON PARK ST]
[San Francisco,94103,400 Block of JESSIE ST]
[San Francisco,94103,1400 Block of MARKET ST]
[San Francisco,94103,3RD ST/JESSIE ST]
[San Francisco,94103,0 Block of 7TH ST]
[San Francisco,94102,500 Block of OFARRELL ST]
[San Francisco,94103,1100 Block of MARKET ST]
[San Francisco,94103,500 Block of STEVENSON ST]
[San Francisco,94102,300 Block of POWELL ST]
[San Francisco,94103,MASON ST/MARKET ST]
[San Francisco,94103,8TH ST/MINNA ST]
[San Francisco,94102,300 Block of FELL ST]
[San Francisco,94102,300 Block of TURK ST]
[San Francisco,94103,MASON ST/MARKET ST]
[San Francisco,94103,0 Block of 4TH ST]
[San Francisco,94103,1100 Block of MARKET ST]
[San Francisco,94102,700 Block 

districtSanFranciscoZipcode: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [19]:
//Reponse 6

fireTSDF.describe("ResponseDelayedinMins").filter($"summary" =!= "stddev").show()


+-------+---------------------+
|summary|ResponseDelayedinMins|
+-------+---------------------+
|  count|               175296|
|   mean|    3.892364154521585|
|    min|          0.016666668|
|    max|              1844.55|
+-------+---------------------+



### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

In [20]:
//Reponse 7-a "MM/dd/yyyy"  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
val annees = fireTSDF.withColumn("Annee", year(to_timestamp($"IncidentDate", "MM/dd/yyyy"))).select("Annee").distinct().orderBy("Annee")
annees.show()

+-----+
|Annee|
+-----+
| 2000|
| 2001|
| 2002|
| 2003|
| 2004|
| 2005|
| 2006|
| 2007|
| 2008|
| 2009|
| 2010|
| 2011|
| 2012|
| 2013|
| 2014|
| 2015|
| 2016|
| 2017|
| 2018|
+-----+



annees: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Annee: int]


### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

In [21]:
//Reponse 7-b: Week: 2018-05-27
val df_week = fireTSDF.withColumn("Annee", year(to_timestamp($"IncidentDate", "MM/dd/yyyy"))).select("IncidentDate","CallNumber","Annee").orderBy("IncidentDate")

df_week.where($"Annee" === "2018").withColumn("week_day",date_sub(next_day($"IncidentDate","sunday"),7)).groupBy("week_day").agg(count("CallNumber").alias("total_appel_week")).orderBy(desc("total_appel_week")).show(1)


+----------+----------------+
|  week_day|total_appel_week|
+----------+----------------+
|2018-05-27|             272|
+----------+----------------+
only showing top 1 row



df_week: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentDate: timestamp, CallNumber: int ... 1 more field]


### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [22]:
//Reponse 8
val df_reponse_quartier = fireTSDF.withColumn("Annee", year(to_timestamp($"IncidentDate", "MM/dd/yyyy"))).select("IncidentDate","CallNumber","Annee","Neighborhood").orderBy("IncidentDate")
df_reponse_quartier.where($"Annee" === "2018").groupBy("Neighborhood").count.orderBy("count").show(false)


+--------------------------+-----+
|Neighborhood              |count|
+--------------------------+-----+
|None                      |5    |
|Lincoln Park              |9    |
|McLaren Park              |14   |
|Seacliff                  |15   |
|Glen Park                 |48   |
|Twin Peaks                |61   |
|Golden Gate Park          |64   |
|Presidio                  |69   |
|Presidio Heights          |71   |
|Treasure Island           |72   |
|Noe Valley                |79   |
|Portola                   |83   |
|Japantown                 |94   |
|Visitacion Valley         |105  |
|Lone Mountain/USF         |108  |
|Inner Richmond            |129  |
|Outer Mission             |137  |
|Oceanview/Merced/Ingleside|139  |
|Haight Ashbury            |140  |
|Russian Hill              |147  |
+--------------------------+-----+
only showing top 20 rows



df_reponse_quartier: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentDate: timestamp, CallNumber: int ... 2 more fields]


### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [23]:
//Reponse 9
// Les DataFrames peuvent être enregistrés sous forme de fichiers Parquet, en conservant les informations du schéma
//fireTSDF.write.parquet("fireTSDF2.parquet")
//val parquetFileDF = spark.read.parquet("fireTSDF2.parquet")
//parquetFileDF.show()
  //convert to parquet
  fireTSDF.write.parquet("fireTSDF.parquet")


### Question 10
**Comment relire les données stockée en format Parquet?**

In [24]:
//Reponse 10
//Exemple de lecture les donnees stockees sous format paquet
// lecture de la colonne Calltype du datafram
val parquetFileDF = spark.read.parquet("fireTSDF.parquet")

// creation d'une vue temporelle et utlisation de requete sql pour projection des donnees
parquetFileDF.createOrReplaceTempView("parquetFile")

// projection des Calltype du dataframe
val callTypeDF = spark.sql("SELECT Calltype FROM parquetFile")
// affichage de quelques calltype
callTypeDF.map(attributes => "CallType: " + attributes(0)).show()

+--------------------+
|               value|
+--------------------+
|CallType: Structu...|
|CallType: Medical...|
|CallType: Medical...|
|CallType: Vehicle...|
|    CallType: Alarms|
|CallType: Structu...|
|    CallType: Alarms|
|    CallType: Alarms|
|CallType: Medical...|
|CallType: Medical...|
|CallType: Medical...|
|CallType: Structu...|
|CallType: Medical...|
|CallType: Medical...|
|CallType: Structu...|
|CallType: Structu...|
|CallType: Structu...|
|CallType: Medical...|
|CallType: Medical...|
|CallType: Medical...|
+--------------------+
only showing top 20 rows



parquetFileDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
callTypeDF: org.apache.spark.sql.DataFrame = [Calltype: string]


## FIN